In [60]:
import sys
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Dropout,Conv1D,MaxPooling1D,initializers,Input
from keras.models import Model
from keras import optimizers,regularizers
from keras.callbacks import History,EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from keras.optimizers import SGD
from keras import regularizers
import nltk
from nltk import pos_tag
from collections import namedtuple
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
from gensim.models import doc2vec
from gensim import models
from gensim.models.doc2vec import TaggedDocument
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from keras import backend as K
from sklearn.model_selection import KFold
from sklearn.datasets import make_classification
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.utils import shuffle, class_weight
from sklearn.decomposition import PCA


In [61]:
with open('task2data.pkl', 'rb') as f:
    data = pickle.load(f)

In [62]:
print(list(data.columns.values))

['sentences', 'article', 'N_sentence', 'is_propaganda', 'sadness', 'joy', 'fear', 'disgust', 'anger']


In [63]:
#data = shuffle(data)

In [82]:
def tokenise(x):
 x=x.lower()
 tokenizer = RegexpTokenizer(r'\w+')
 x=(tokenizer.tokenize(x))
 filtered_words =([word for word in x if word not in stopwords.words('english')])
 filtered_words=" ".join(filtered_words)
 st = PorterStemmer()
 stemmed_words = ' '.join(([st.stem(word) for word in filtered_words]))
 return stemmed_words

In [83]:
data["new_sentences"] = data["sentences"].apply(lambda x: tokenise(x))

In [84]:
tag_values=[]
docs=[]
y= np.array([])
for index, row in data.iterrows():
 w=row["new_sentences"].split()
 word_value=row["new_sentences"].split()
 t1=[i]
 all_tags=nltk.pos_tag(row["sentences"].split())
 tag_values.append(all_tags)
 #t=[row["sadness"],row["joy"],row["fear"],row["disgust"],row["anger"]]
 #val=[i[1] for i in t1]
 docs.append(TaggedDocument(words=w, tags=t1))
 if(row["is_propaganda"]=="propaganda"):
        y=np.append(y, 0)
 else:
    y=np.append(y, 1)

In [85]:
#including grammatical features nouns, pronouns, etc
pos_tags=['CC','CD','DT','EX','FW','IN','JJ','JJR','JJS','LS','MD','NN','NNS','NNP','NNPS','PDT','POS','PRP','PRP$','RB','RBR','RBS','RP','TO','UH','VB','VBD','VBG','VBN','VBP','VBZ','WDT','WP','WP$','WRB']
tagm = [[sum(sen,()).count(tag) for tag in pos_tags] for sen in tag_values]
tagm=np.array(tagm)
print(tagm.shape)

(14263, 35)


In [86]:
size=10
model = models.Doc2Vec(vector_size=size, window=3, min_count=2, workers=4, epochs=20)
model.build_vocab(docs)

In [87]:
model.train(docs, total_examples=model.corpus_count, epochs=model.iter)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


In [89]:
y=y.reshape(-1, 1)
x=np.zeros((y.shape[0], size))
for i in range(y.shape[0]):
    x[i]=model[i]

KeyError: "tag '293' not seen in training corpus/invalid"

In [91]:
count_0=np.sum(y==0)
count_1=np.sum(y==1)

print(count_0/y.shape[0])
print(count_1/y.shape[0])

0.27609899740587535
0.7239010025941247


In [92]:
values=np.array(data.iloc[:,4:9])
values2=np.array(data.iloc[:,2:3])
x=np.append(x, values, axis=1)
x=np.append(x, values2,axis=1)
x=np.append(x, tagm,axis=1)
print(x.shape)

(14263, 51)


In [93]:
split_index=int(y.shape[0]*0.8)
y_train=y[0:split_index, :]
x_train=x[0:split_index,:]
y_val=y[split_index:,:]
x_val=x[split_index:,:]

print(x_train.shape)
print(x_val.shape)

data_train= data.iloc[0:split_index,0:1]
data_val=data.iloc[split_index:,0:1]

(11410, 51)
(2853, 51)


In [94]:
count_0_train=np.sum(y_train==0)
count_1_train=np.sum(y_train==1)

print(count_0_train/y_train.shape[0])
print(count_1_train/y_train.shape[0])

0.28089395267309375
0.7191060473269062


In [96]:
count_0=np.sum(y_val==0)
count_1=np.sum(y_val==1)

print(count_0/y_val.shape[0])
print(count_1/y_val.shape[0])

0.25692253767963547
0.7430774623203645
a


In [97]:
#Using Logistic Regression as baseline
pca = PCA(n_components=20)
pca.fit(x_train)
new_x_train=pca.transform(x_train)
new_x_val=pca.transform(x_val)

logReg = LogisticRegression()
logReg.fit(new_x_train, y_train)
print("With PCA")
print(logReg.score(new_x_train, y_train)) #Accuracy on train

#on val
print(logReg.score(new_x_val,y_val))
print("Without PCA")
logReg.fit(x_train, y_train)
print(logReg.score(x_train, y_train))
print(logReg.score(x_val, y_val))

With PCA
0.7293602103418054
0.745881528215913
Without PCA
0.7382997370727432
0.7511391517700666


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape o

In [ ]:
clf = svm.SVC(kernel='linear', C = 1.0)
clf.fit(x_train,y_train)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [291]:
print(clf.score(x_train,y_train)) #Training score
print(clf.score(x_val,y_val)) #validation score

0.7146434294871795


In [226]:
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    # tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [98]:

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1(y_true,y_pred):
   precisions = precision(y_true, y_pred)
   recalls = recall(y_true, y_pred)
   return 2*((precisions*recalls)/(precisions+recalls+K.epsilon()))

In [99]:
y_train=y_train.reshape((y_train.shape[0],))
print(y_train.shape)
class_weights_values= class_weight.compute_class_weight('balanced',np.unique(y_train), y_train)
y_train=y_train.reshape(-1, 1)

(11410,)


In [158]:
kfold = KFold(n_splits=10, shuffle=True)
accuracy=[]
f1_value=[]
for train, test in kfold.split(x_train,y_train):
 model = Sequential()
 model.add(Dense(175, input_dim=x_train.shape[1], activation='relu'))
 model.add(Dropout(0.6))
 model.add(Dense(175, activation='relu'))
 model.add(Dropout(0.5))
 model.add(Dense(1, activation='sigmoid'))
 model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['mse','acc',f1])
 train=model.fit(x_train[train], y_train[train],epochs=500, verbose=2,batch_size=x_train.shape[0])
 train_model=model.evaluate(x_train[test],y_train[test],verbose=2)
 accuracy.append(train_model[2])
 f1_value.append(train_model[3])

Epoch 1/500
 - 2s - loss: 10.1051 - mean_squared_error: 0.6275 - acc: 0.3725 - f1: 0.3159
Epoch 2/500
 - 0s - loss: 10.3123 - mean_squared_error: 0.6404 - acc: 0.3596 - f1: 0.2920
Epoch 3/500
 - 0s - loss: 10.2225 - mean_squared_error: 0.6348 - acc: 0.3652 - f1: 0.3048
Epoch 4/500
 - 0s - loss: 10.0756 - mean_squared_error: 0.6257 - acc: 0.3743 - f1: 0.3163
Epoch 5/500
 - 0s - loss: 10.3188 - mean_squared_error: 0.6409 - acc: 0.3591 - f1: 0.3029
Epoch 6/500
 - 0s - loss: 10.2554 - mean_squared_error: 0.6369 - acc: 0.3631 - f1: 0.3016
Epoch 7/500
 - 0s - loss: 10.1907 - mean_squared_error: 0.6329 - acc: 0.3671 - f1: 0.3125
Epoch 8/500
 - 0s - loss: 10.2240 - mean_squared_error: 0.6349 - acc: 0.3651 - f1: 0.3052
Epoch 9/500
 - 0s - loss: 10.2740 - mean_squared_error: 0.6380 - acc: 0.3620 - f1: 0.3019
Epoch 10/500
 - 0s - loss: 10.0645 - mean_squared_error: 0.6250 - acc: 0.3750 - f1: 0.3185
Epoch 11/500
 - 0s - loss: 10.1536 - mean_squared_error: 0.6305 - acc: 0.3695 - f1: 0.3118
Epoch 12

 - 0s - loss: 10.2059 - mean_squared_error: 0.6338 - acc: 0.3662 - f1: 0.3013
Epoch 92/500
 - 0s - loss: 10.2412 - mean_squared_error: 0.6360 - acc: 0.3640 - f1: 0.3040
Epoch 93/500
 - 0s - loss: 10.1652 - mean_squared_error: 0.6312 - acc: 0.3688 - f1: 0.3050
Epoch 94/500
 - 0s - loss: 10.1507 - mean_squared_error: 0.6303 - acc: 0.3697 - f1: 0.3095
Epoch 95/500
 - 0s - loss: 10.2372 - mean_squared_error: 0.6357 - acc: 0.3643 - f1: 0.2988
Epoch 96/500
 - 0s - loss: 10.1754 - mean_squared_error: 0.6319 - acc: 0.3681 - f1: 0.3108
Epoch 97/500
 - 0s - loss: 10.2509 - mean_squared_error: 0.6366 - acc: 0.3634 - f1: 0.3003
Epoch 98/500
 - 0s - loss: 10.1848 - mean_squared_error: 0.6325 - acc: 0.3675 - f1: 0.3097
Epoch 99/500
 - 0s - loss: 10.1399 - mean_squared_error: 0.6297 - acc: 0.3703 - f1: 0.3096
Epoch 100/500
 - 0s - loss: 10.2934 - mean_squared_error: 0.6392 - acc: 0.3608 - f1: 0.2948
Epoch 101/500
 - 0s - loss: 10.2756 - mean_squared_error: 0.6381 - acc: 0.3619 - f1: 0.3013
Epoch 102/

Epoch 181/500
 - 0s - loss: 10.3632 - mean_squared_error: 0.6436 - acc: 0.3564 - f1: 0.2932
Epoch 182/500
 - 0s - loss: 10.2102 - mean_squared_error: 0.6340 - acc: 0.3660 - f1: 0.3043
Epoch 183/500
 - 0s - loss: 10.2540 - mean_squared_error: 0.6368 - acc: 0.3632 - f1: 0.3001
Epoch 184/500
 - 0s - loss: 10.1288 - mean_squared_error: 0.6290 - acc: 0.3710 - f1: 0.3119
Epoch 185/500
 - 0s - loss: 10.2227 - mean_squared_error: 0.6348 - acc: 0.3652 - f1: 0.3033
Epoch 186/500
 - 0s - loss: 10.2093 - mean_squared_error: 0.6339 - acc: 0.3661 - f1: 0.2988
Epoch 187/500
 - 0s - loss: 10.1828 - mean_squared_error: 0.6324 - acc: 0.3676 - f1: 0.3134
Epoch 188/500
 - 0s - loss: 10.2616 - mean_squared_error: 0.6373 - acc: 0.3627 - f1: 0.3020
Epoch 189/500
 - 0s - loss: 10.1069 - mean_squared_error: 0.6276 - acc: 0.3724 - f1: 0.3133
Epoch 190/500
 - 0s - loss: 10.2051 - mean_squared_error: 0.6338 - acc: 0.3662 - f1: 0.3078
Epoch 191/500
 - 0s - loss: 10.2377 - mean_squared_error: 0.6358 - acc: 0.3642 -

 - 0s - loss: 10.1603 - mean_squared_error: 0.6309 - acc: 0.3691 - f1: 0.3068
Epoch 271/500
 - 0s - loss: 10.1317 - mean_squared_error: 0.6292 - acc: 0.3708 - f1: 0.3133
Epoch 272/500
 - 0s - loss: 10.1981 - mean_squared_error: 0.6333 - acc: 0.3667 - f1: 0.3022
Epoch 273/500
 - 0s - loss: 10.1504 - mean_squared_error: 0.6303 - acc: 0.3697 - f1: 0.3120
Epoch 274/500
 - 0s - loss: 10.2039 - mean_squared_error: 0.6337 - acc: 0.3663 - f1: 0.3053
Epoch 275/500
 - 0s - loss: 10.3051 - mean_squared_error: 0.6400 - acc: 0.3600 - f1: 0.3007
Epoch 276/500
 - 0s - loss: 10.1490 - mean_squared_error: 0.6302 - acc: 0.3698 - f1: 0.3113
Epoch 277/500
 - 0s - loss: 10.2428 - mean_squared_error: 0.6361 - acc: 0.3639 - f1: 0.3036
Epoch 278/500
 - 0s - loss: 10.1724 - mean_squared_error: 0.6317 - acc: 0.3683 - f1: 0.3100
Epoch 279/500
 - 0s - loss: 10.2337 - mean_squared_error: 0.6355 - acc: 0.3645 - f1: 0.3023
Epoch 280/500
 - 0s - loss: 10.3170 - mean_squared_error: 0.6407 - acc: 0.3593 - f1: 0.2908
Ep

Epoch 360/500
 - 0s - loss: 10.1869 - mean_squared_error: 0.6326 - acc: 0.3674 - f1: 0.3051
Epoch 361/500
 - 0s - loss: 10.2432 - mean_squared_error: 0.6361 - acc: 0.3639 - f1: 0.2999
Epoch 362/500
 - 0s - loss: 10.1956 - mean_squared_error: 0.6332 - acc: 0.3668 - f1: 0.3099
Epoch 363/500
 - 0s - loss: 10.1100 - mean_squared_error: 0.6278 - acc: 0.3722 - f1: 0.3133
Epoch 364/500
 - 0s - loss: 10.1583 - mean_squared_error: 0.6308 - acc: 0.3692 - f1: 0.3114
Epoch 365/500
 - 0s - loss: 10.2132 - mean_squared_error: 0.6342 - acc: 0.3658 - f1: 0.3054
Epoch 366/500
 - 0s - loss: 10.3100 - mean_squared_error: 0.6403 - acc: 0.3597 - f1: 0.2982
Epoch 367/500
 - 0s - loss: 10.1131 - mean_squared_error: 0.6280 - acc: 0.3720 - f1: 0.3137
Epoch 368/500
 - 0s - loss: 10.2142 - mean_squared_error: 0.6343 - acc: 0.3657 - f1: 0.3095
Epoch 369/500
 - 0s - loss: 10.2572 - mean_squared_error: 0.6370 - acc: 0.3630 - f1: 0.2992
Epoch 370/500
 - 0s - loss: 10.2279 - mean_squared_error: 0.6352 - acc: 0.3648 -

KeyboardInterrupt: 

In [155]:
print("For the training set")
print("Accuracy is: "+str(np.mean(accuracy)))
print("F1 is: "+str(np.mean(f1_value)))

For the training set
Accuracy is: 0.5392638038925492
F1 is: 0.49795125637735643


In [157]:
print("For the test test")
test_model=model.evaluate(x_val,y_val,verbose=2)
print("Accuracy is: "+str(test_model[2]))
print("F1 is: "+ str(test_model[3]))

For the test test
Accuracy is: 0.7430774623203645
F1 is: 0.8451232763077141


In [112]:
model = Sequential()
model.add(Dense(175, input_dim=x_train.shape[1], activation='relu',activity_regularizer=regularizers.l2(0.01)))
model.add(Dense(175, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
sgd=optimizers.SGD(lr=0.080, momentum=0.0, decay=0.0, nesterov=False)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['mse','acc',f1,precision,recall])
train=model.fit(x_train, y_train,epochs=10, verbose=1,class_weight=class_weights_values,batch_size=x_train.shape[0])
train_model=model.evaluate(x_train,y_train,verbose=2)

Epoch 1/10
11410/11410 [==============================] - 1s 49us/step - loss: 439484.7188 - mean_squared_error: 0.2226 - acc: 0.6917 - f1: 0.8107 - precision: 0.7258 - recall: 0.9180
Epoch 2/10
11410/11410 [==============================] - 0s 6us/step - loss: 0.6976 - mean_squared_error: 0.2492 - acc: 0.7191 - f1: 0.8366 - precision: 0.7192 - recall: 0.9998
Epoch 3/10
11410/11410 [==============================] - 0s 5us/step - loss: 0.6891 - mean_squared_error: 0.2457 - acc: 0.7191 - f1: 0.8366 - precision: 0.7191 - recall: 1.0000
Epoch 4/10
11410/11410 [==============================] - 0s 5us/step - loss: 0.6816 - mean_squared_error: 0.2424 - acc: 0.7191 - f1: 0.8366 - precision: 0.7191 - recall: 1.0000
Epoch 5/10
11410/11410 [==============================] - 0s 5us/step - loss: 0.6748 - mean_squared_error: 0.2393 - acc: 0.7191 - f1: 0.8366 - precision: 0.7191 - recall: 1.0000
Epoch 6/10
11410/11410 [==============================] - 0s 6us/step - loss: 0.6687 - mean_squared_erro

In [113]:
#For training
print("For training \n")
print("Precision: "+str(train_model[4]))
print("Recall: "+str(train_model[5]))
print("F1: "+str(train_model[3]))
print("Accuracy: "+str(train_model[2]))

For training 

Precision: 0.7191060473373541
Recall: 1.0
F1: 0.82643346677842
Accuracy: 0.7191060473373541


In [114]:
val_model=model.evaluate(x_val,y_val,verbose=2)

In [115]:
print("For validation \n")
print("Precision: "+str(val_model[4]))
print("Recall: "+str(val_model[5]))
print("F1: "+str(val_model[3]))
print("Accuracy: "+str(val_model[2]))

For validation 

Precision: 0.7430774623203645
Recall: 1.0
F1: 0.8451232763077141
Accuracy: 0.7430774623203645


In [116]:
#using LSTM
new_x_train=x_train
new_x_train= new_x_train.reshape((new_x_train.shape[0],1,new_x_train.shape[1]))
print(new_x_train.shape)

(11410, 1, 51)


In [117]:
new_x_val=x_val
new_x_val= new_x_val.reshape((new_x_val.shape[0],1,new_x_val.shape[1]))
print(new_x_val.shape)

(2853, 1, 51)


In [123]:
model = Sequential()
model.add(LSTM(175,input_shape=(1,x.shape[1]),return_sequences=True,activation='relu'))
model.add(Dropout(0.6))
model.add(Flatten())
model.add(Dense(175, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(1, activation='sigmoid'))
#es=EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=2, mode='auto')
model.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['mse','acc',f1,precision,recall])
train=model.fit(new_x_train, y_train,epochs=60, verbose=2,class_weight=class_weights_values,batch_size=x_train.shape[0],validation_data=(new_x_val, y_val))
train_model=model.evaluate(new_x_train,y_train,verbose=2)

Train on 11410 samples, validate on 2853 samples
Epoch 1/60
 - 2s - loss: 1.3939 - mean_squared_error: 0.3663 - acc: 0.3975 - f1: 0.3958 - precision: 0.7095 - recall: 0.2745 - val_loss: 0.9217 - val_mean_squared_error: 0.3237 - val_acc: 0.4791 - val_f1: 0.5900 - val_precision: 0.7108 - val_recall: 0.5042
Epoch 2/60
 - 0s - loss: 1.1471 - mean_squared_error: 0.3231 - acc: 0.4938 - f1: 0.5842 - precision: 0.7136 - recall: 0.4945 - val_loss: 0.7095 - val_mean_squared_error: 0.2549 - val_acc: 0.6200 - val_f1: 0.7517 - val_precision: 0.7306 - val_recall: 0.7741
Epoch 3/60
 - 0s - loss: 0.9488 - mean_squared_error: 0.2847 - acc: 0.5767 - f1: 0.6940 - precision: 0.7226 - recall: 0.6676 - val_loss: 0.6105 - val_mean_squared_error: 0.2093 - val_acc: 0.7392 - val_f1: 0.8499 - val_precision: 0.7424 - val_recall: 0.9939
Epoch 4/60
 - 0s - loss: 0.8846 - mean_squared_error: 0.2679 - acc: 0.6215 - f1: 0.7464 - precision: 0.7202 - recall: 0.7745 - val_loss: 0.5853 - val_mean_squared_error: 0.1975 - v

Epoch 33/60
 - 0s - loss: 0.6094 - mean_squared_error: 0.2057 - acc: 0.7055 - f1: 0.8235 - precision: 0.7236 - recall: 0.9555 - val_loss: 0.5545 - val_mean_squared_error: 0.1847 - val_acc: 0.7434 - val_f1: 0.8527 - val_precision: 0.7437 - val_recall: 0.9991
Epoch 34/60
 - 0s - loss: 0.6072 - mean_squared_error: 0.2051 - acc: 0.7089 - f1: 0.8255 - precision: 0.7254 - recall: 0.9576 - val_loss: 0.5539 - val_mean_squared_error: 0.1845 - val_acc: 0.7424 - val_f1: 0.8520 - val_precision: 0.7434 - val_recall: 0.9976
Epoch 35/60
 - 0s - loss: 0.6100 - mean_squared_error: 0.2051 - acc: 0.7080 - f1: 0.8251 - precision: 0.7247 - recall: 0.9578 - val_loss: 0.5525 - val_mean_squared_error: 0.1840 - val_acc: 0.7427 - val_f1: 0.8521 - val_precision: 0.7437 - val_recall: 0.9976
Epoch 36/60
 - 0s - loss: 0.6049 - mean_squared_error: 0.2039 - acc: 0.7093 - f1: 0.8266 - precision: 0.7238 - recall: 0.9634 - val_loss: 0.5533 - val_mean_squared_error: 0.1843 - val_acc: 0.7441 - val_f1: 0.8528 - val_precisi

In [124]:
print("For training \n")
print("Precision: "+str(train_model[4]))
print("Recall: "+str(train_model[5]))
print("F1: "+str(train_model[3]))
print("Accuracy: "+str(train_model[2]))

For training 

Precision: 0.7242916825254375
Recall: 0.9907047804983743
F1: 0.8270446840866317
Accuracy: 0.7234005258649614


In [125]:
val_model=model.evaluate(new_x_val,y_val,verbose=2)

In [126]:
print("For validation \n")
print("Precision: "+str(val_model[4]))
print("Recall: "+str(val_model[5]))
print("F1: "+str(val_model[3]))
print("Accuracy: "+str(val_model[2]))

For validation 

Precision: 0.7481575139229899
Recall: 0.989601422726711
F1: 0.8444051558704824
Accuracy: 0.7441289870311952


In [133]:
vocabulary_size = 1200
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(data_train['sentences'])
sequences = tokenizer.texts_to_sequences(data_train['sentences'])
data_words_train = pad_sequences(sequences, maxlen=50)
#data_words_train=np.append(data_words_train, tagm,axis=1)

In [135]:
#using LSTM for word embeddings
model = Sequential()
model.add(Embedding(1200, 100, input_length=50))
model.add(Dropout(0.6))
model.add(Flatten())
model.add(Dense(175, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
#es=EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=2, mode='auto')
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['mse','acc',f1,precision,recall])
train=model.fit(data_words_train, y_train,epochs=30, verbose=2,class_weight=class_weights_values,batch_size=x_train.shape[0])
train_model=model.evaluate(data_words_train,y_train,verbose=2)

Epoch 1/30
 - 3s - loss: 0.6863 - mean_squared_error: 0.2466 - acc: 0.5735 - f1: 0.6872 - precision: 0.7271 - recall: 0.6514
Epoch 2/30
 - 1s - loss: 0.7852 - mean_squared_error: 0.2345 - acc: 0.7200 - f1: 0.8369 - precision: 0.7200 - recall: 0.9990
Epoch 3/30
 - 1s - loss: 0.6080 - mean_squared_error: 0.2084 - acc: 0.7243 - f1: 0.8380 - precision: 0.7255 - recall: 0.9918
Epoch 4/30
 - 1s - loss: 0.5732 - mean_squared_error: 0.1933 - acc: 0.7232 - f1: 0.8376 - precision: 0.7244 - recall: 0.9928
Epoch 5/30
 - 2s - loss: 0.5705 - mean_squared_error: 0.1922 - acc: 0.7247 - f1: 0.8382 - precision: 0.7259 - recall: 0.9917
Epoch 6/30
 - 1s - loss: 0.5664 - mean_squared_error: 0.1907 - acc: 0.7258 - f1: 0.8387 - precision: 0.7267 - recall: 0.9916
Epoch 7/30
 - 1s - loss: 0.5647 - mean_squared_error: 0.1899 - acc: 0.7266 - f1: 0.8390 - precision: 0.7276 - recall: 0.9907
Epoch 8/30
 - 1s - loss: 0.5610 - mean_squared_error: 0.1886 - acc: 0.7272 - f1: 0.8393 - precision: 0.7279 - recall: 0.9911


In [137]:
print("For training \n")
print("Precision: "+str(train_model[4]))
print("Recall: "+str(train_model[5]))
print("F1: "+str(train_model[3]))
print("Accuracy: "+str(train_model[2]))

For training 

Precision: 0.7594832291632342
Recall: 0.9853250294584229
F1: 0.8501454054734667
Accuracy: 0.7707274320353341


In [138]:
vocabulary_size = 1200
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(data_val['sentences'])
sequences = tokenizer.texts_to_sequences(data_val['sentences'])
data_words_val = pad_sequences(sequences, maxlen=50)


In [139]:
val_model=model.evaluate(data_words_val,y_val,verbose=2)

In [140]:
print("For validation \n")
print("Precision: "+str(val_model[4]))
print("Recall: "+str(val_model[5]))
print("F1: "+str(val_model[3]))
print("Accuracy: "+str(val_model[2]))

For validation 

Precision: 0.7501297164205664
Recall: 0.9583619482426153
F1: 0.8333367870182394
Accuracy: 0.7308096740273397


In [141]:
#Using CNN for word embeddings
model_conv = Sequential()
model_conv.add(Embedding(vocabulary_size, 100, input_length=50))
model_conv.add(Dropout(0.5))
model_conv.add(Conv1D(175, 5, activation='relu'))
model_conv.add(MaxPooling1D(pool_size=4))
model_conv.add(LSTM(175,activation='relu'))
model_conv.add(Dense(1, activation='sigmoid'))
model_conv.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy',f1,precision,recall])
train=model_conv.fit(data_words_train, y_train, epochs=3,class_weight=class_weights_values)
train_model=model_conv.evaluate(data_words_train,y_train,verbose=2)

Epoch 1/3
11410/11410 [==============================] - 14s 1ms/step - loss: 0.5716 - acc: 0.7208 - f1: 0.8301 - precision: 0.7285 - recall: 0.9733
Epoch 2/3
11410/11410 [==============================] - 12s 1ms/step - loss: 0.5281 - acc: 0.7473 - f1: 0.8401 - precision: 0.7644 - recall: 0.9400
Epoch 3/3
11410/11410 [==============================] - 12s 1ms/step - loss: 0.5078 - acc: 0.7630 - f1: 0.8480 - precision: 0.7773 - recall: 0.9401


In [34]:
print("For training \n")
print("Precision: "+str(train_model[3]))
print("Recall: "+str(train_model[4]))
print("F1: "+str(train_model[2]))
print("Accuracy: "+str(train_model[1]))

For training 

Precision: 0.7593991969156223
Recall: 0.973009341354437
F1: 0.8463087762060341
Accuracy: 0.7668711656023807


In [36]:
val_model=model_conv.evaluate(data_words_val,y_val,verbose=2)

In [38]:
print("For testing \n")
print("Precision: "+str(val_model[3]))
print("Recall: "+str(val_model[4]))
print("F1: "+str(val_model[2]))
print("Accuracy: "+str(val_model[1]))

For testing 

Precision: 0.7463187756891047
Recall: 0.9503596593654243
F1: 0.8280860715024395
Accuracy: 0.722397476340694
